<h1><center> $$Un tableau de bord sur le Covid$$ </center></h1>


Contexte du projet

Maintenant que vous êtes réputés pour vos dashboards, La haute autorité de santé vous commande un tableau de bord en ligne pour pouvoir suivre les indicateurs sur le Covid-19.

Modalités pédagogiques

Le projet est a réaliser en équipe de 2 ou 3 apprenants. Vous devez coder un dashboard à partir des datasets proposé sur le Covid-19. Vous aurez sans doute à retraiter ces données pour les rendres exploitables. En Front, affichez dans votre Dashboard 3 à 4 indicateurs qui vous parraissent intéressants.

# PREPARATION DES DONNEES

In [41]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy

In [42]:
#source https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/
df= pd.read_csv('./donneeshospitalieres.csv', sep = ";", na_filter=False)
df

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6
...,...,...,...,...,...,...,...
93925,974,1,2021-01-21,26,6,371,26
93926,974,2,2021-01-21,20,0,381,28
93927,976,0,2021-01-21,13,4,568,44
93928,976,1,2021-01-21,4,3,275,27


Nous récupérons quotidiennement par département grâce au dataframe les informations suivantes: 

    le nombre de patients hospitalisés, 
    
    le nombre de patients actuellement en réanimation , 
    
    le nombre de patients retourné à domicile
    
    le nombre de décès.
    
Chaque département , affiche les données par sexe(=>3 lignes par département).
    1=> homme
    2=> femme
    0=> total

Notre jeu de données comporte les informations suivantes :    
    
    Les données hospitalières relatives à l'épidémie du COVID-19:

            dep = par département 

            sexe =sexe du patient 

            hosp :nombre de patients hospitalisés 

            rea : nombre de personnes actuellement en réanimation ou soins intensifs

            rad : nombre cumulé de personnes retournées à domicile

            dc : nombre cumulé de personnes décédées.

In [53]:
print(list(df), end="\n\n")
for var in list(df):
    print("{} contains {} labels".format(var,len(df[var].unique())), end="\n\n")

['level_0', 'index', 'dep', 'sexe', 'jour', 'hosp', 'rea', 'rad', 'dc']

level_0 contains 93930 labels

index contains 93930 labels

dep contains 101 labels

sexe contains 3 labels

jour contains 310 labels

hosp contains 1502 labels

rea contains 426 labels

rad contains 5393 labels

dc contains 1842 labels



on change le format des 10 premiers départements

In [43]:
df['dep'] = df['dep'] .replace(['1','2','3', '4', '5', '6', '7', '8', '9'],['01','02','03', '04', '05', '06', '07', '08', '09'])

In [46]:
df.reset_index(level=0, inplace=True)

In [48]:
df.to_csv('d5.csv',index=False)

2eme fichier données géographique pour compléter nos informations, et population en FRANCE

In [56]:
coordonnees= pd.read_csv('./dep_coor.csv', sep = ",", na_filter=False)
coordonnees

,dep_code,dep_name,region_code,region,Longitude en degre,Latitude en degre,Pop_2018
0,1,Ain,84,Auvergne-Rhône-Alpes,5.340247,46.080770,663720
1,2,Aisne,32,Hauts-de-France,3.530170,49.547387,545353
2,3,Allier,84,Auvergne-Rhône-Alpes,3.179084,46.352009,346156
3,4,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur,6.149504,44.077202,168613
4,5,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur,6.115795,44.578924,145309
...,...,...,...,...,...,...,...
97,972,Martinique,2,Martinique,13.469529,-51.579534,373762
98,973,Guyane,3,Guyane,4.673780,-52.996532,278472
99,974,La Réunion,4,La Réunion,-10.299767,37.345128,865507
100,975,Saint-Pierre-et-Miquelon,COM,Collectivités d'Outre-Mer,46.710700,1.718190,6224


In [59]:
data = pd.merge(left=coordonnees, right=df, left_on='dep_code', right_on='dep')
data

,dep_code,dep_name,region_code,region,Longitude en degre,Latitude en degre,Pop_2018,level_0,index,dep,sexe,jour,hosp,rea,rad,dc
0,10,Aube,44,Grand Est,4.181066,48.301786,317139,27,27,10,0,2020-03-18,5,0,0,0
1,10,Aube,44,Grand Est,4.181066,48.301786,317139,28,28,10,1,2020-03-18,4,0,0,0
2,10,Aube,44,Grand Est,4.181066,48.301786,317139,29,29,10,2,2020-03-18,1,0,0,0
3,10,Aube,44,Grand Est,4.181066,48.301786,317139,330,330,10,0,2020-03-19,6,1,0,0
4,10,Aube,44,Grand Est,4.181066,48.301786,317139,331,331,10,1,2020-03-19,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85555,976,Mayotte,6,Mayotte,45.146959,-12.816065,262895,93625,93625,976,1,2021-01-20,4,2,274,27
85556,976,Mayotte,6,Mayotte,45.146959,-12.816065,262895,93626,93626,976,2,2021-01-20,7,1,287,17
85557,976,Mayotte,6,Mayotte,45.146959,-12.816065,262895,93927,93927,976,0,2021-01-21,13,4,568,44
85558,976,Mayotte,6,Mayotte,45.146959,-12.816065,262895,93928,93928,976,1,2021-01-21,4,3,275,27


In [60]:
data.to_csv('d6.csv',index=False)